In [17]:
# Loading the Libraries
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os,glob

In [18]:
AGE_GROUP_PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/age_groups_test.csv'
GEO_NAME_PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/geo_name_test.csv'
#CHANGE
age_group = pd.read_csv(AGE_GROUP_PATH)


In [19]:
age_group.head()

,dguid,male_0_to_4_years,male_10_to_14_years,male_100_years_and_over,male_15_to_19_years,male_20_to_24_years,male_25_to_29_years,male_30_to_34_years,male_35_to_39_years,male_40_to_44_years,...,female_50_to_54_years,female_55_to_59_years,female_60_to_64_years,female_65_to_69_years,female_70_to_74_years,female_75_to_79_years,female_80_to_84_years,female_85_to_89_years,female_90_to_94_years,female_95_to_99_years
0,2021A000210,9760.0,13535.0,15.0,13275.0,13555.0,12610.0,13520.0,14205.0,14775.0,...,19685.0,21615.0,21830.0,20735.0,17805.0,11780.0,7280.0,3910.0,1880.0,485.0
1,2021A000211,3475.0,4455.0,5.0,4335.0,5565.0,4765.0,4315.0,4300.0,4360.0,...,5115.0,6005.0,5970.0,5450.0,4770.0,3095.0,2110.0,1355.0,640.0,210.0
2,2021A000212,20610.0,25505.0,50.0,24770.0,30115.0,30375.0,29295.0,27960.0,26900.0,...,33175.0,39870.0,39850.0,35665.0,31140.0,20925.0,14040.0,8740.0,4300.0,1435.0
3,2021A000213,16625.0,20755.0,40.0,20105.0,21995.0,21670.0,21670.0,22470.0,23095.0,...,26450.0,31795.0,31870.0,29605.0,25135.0,16580.0,11310.0,7120.0,3645.0,1200.0
4,2021A000224,217145.0,250855.0,330.0,219400.0,235250.0,266455.0,267770.0,274350.0,286405.0,...,260325.0,308465.0,317900.0,277950.0,237910.0,177050.0,119580.0,81150.0,44535.0,11790.0


In [20]:
age_group = age_group.fillna(0)

In [21]:
# Local Connection
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + "root" + ':' + "Chuckie2020!" + '@' + "localhost" + '/' + "canada_vis" 

engine = create_engine(conn_string, connect_args=ssl_args)

with engine.begin() as connection:
    age_group.to_sql('age_groups', con=connection, if_exists='replace')


In [22]:
# Connecting to Planet Scale
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + os.getenv("USERNAME") + ':' + os.getenv("PASSWORD") + '@' + os.getenv("HOST") + '/' + os.getenv("DATABASE") 

engine = create_engine(conn_string, connect_args=ssl_args)

with engine.begin() as connection:
    age_group.to_sql('age_groups', con=connection, if_exists='replace')

In [27]:
with engine.begin() as connection:
    read_test = pd.read_sql("SELECT summary.dguid, geo_names.parent_dguid, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY summary.total_population_2021 DESC) AS pop_rank, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY total_population_percentage_change_2016_to_2021 DESC) AS pop_growth_rank, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY total_population_density_per_square_kilometre DESC) AS pop_density_rank, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY total_average_household_size DESC) AS household_size_rank, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY total_median_age_of_the_population DESC) AS age_rank, RANK () OVER (PARTITION BY geo_names.parent_dguid ORDER BY total_median_employment_income_in_2020_among_recipients DESC) AS income_rank FROM summary INNER JOIN geo_names ON summary.dguid = geo_names.dguid", con=connection)
#     df = pd.read_csv(csv,encoding="ISO-8859-1")
#     df.to_sql('age_groups', con=connection, if_exists='append')
    read_test.to_sql('ranks', con=connection, if_exists='replace')



In [ ]:
# Upload Census Division and Province Data
# with engine.begin() as connection:
#     new_df.to_sql('age_groups', con=connection, if_exists='append')